# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

---

# Armado de ventanas

In [3]:
for ventana_nro in range(1,6):
    auctions["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (19+ventana_nro)))

---

# Creación de features

## Creación de sets de entrenamiento

In [4]:
Xs = {}
for ventana_nro in range(1,6):
    Xs[ventana_nro] = auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"].drop_duplicates().to_frame().set_index("device_id")

## Función generadora de features

In [5]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    for ventana_nro in range(1, 6):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)
        return None

---

## Cantidad de apariciones en subastas dentro de la ventana

In [6]:
def cantidad_apariciones_en_subastas(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_apariciones"})

In [7]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_en_subastas, Xs)

---

## Cantidad de apariciones discriminando períodos

In [8]:
# Creo columnas para luego separar
auctions['hora'] = auctions['date'].dt.hour
auctions['dia'] = auctions['date'].dt.day
auctions['dia_de_la_semana'] = auctions['date'].dt.dayofweek

In [9]:
def cantidad_apariciones_por_hora(dataframe, nro_ventana):
    apariciones_por_hora = dataframe.groupby(["device_id", "hora"]).agg(({"date" : "count"}))
    apariciones_por_hora = apariciones_por_hora.unstack().fillna(0).astype(np.int64)
    apariciones_por_hora.columns = ["apariciones_en_hora_{}".format(hora) for hora in range(0,24)]
    return apariciones_por_hora

In [10]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_por_hora, Xs)

In [11]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    dias = dataframe["dia"].drop_duplicates
    apariciones_por_dia = dataframe.groupby(["device_id", "dia"]).agg(({"date" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["apariciones_en_dia_{}".format(dia) for dia in range(1,4)]
    return apariciones_por_dia

In [12]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_por_dia, Xs)

## Entrenamiento: [18/4 , 20/4] ; Target: [21/4 , 23/4]

---

## Entrenamiento: [19/4 , 21/4] ; Target: [22/4 , 24/4]

---

## Entrenamiento: [20/4 , 22/4] ; Target: [23/4 , 25/4]

---

## Entrenamiento: [21/4 , 23/4] ; Target: [24/4 , 26/4]

---

## Entrenamiento: [22/4 , 24/4] ; Target: [25/4 , 27/4]

---